In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

### Load the data

In [2]:
data = pd.read_csv("../data/alzheimers_disease_data.csv")
data = data.drop("DoctorInCharge", axis=1) # this attribute is confidential in the data, and thus not useful 
data.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,...,6.518877,0,0,1.725883,0,0,0,1,0,0
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,...,7.118696,0,0,2.592424,0,0,0,0,1,0
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,...,5.895077,0,0,7.119548,0,1,0,1,0,0
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,...,8.965106,0,1,6.481226,0,0,0,0,0,0
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,...,6.045039,0,0,0.014691,0,0,1,1,0,0


### Undersampling by ethnicity

In [3]:
e = data["Ethnicity"]
print('Original dataset shape %s' % Counter(e))

ethnicity_counts = dict(data["Ethnicity"].value_counts())
num_ethnicities = len(ethnicity_counts)
min_count = min(ethnicity_counts.values())

strategy_under = {ethnicity: min_count for ethnicity in range(num_ethnicities)}
under = RandomUnderSampler(sampling_strategy=strategy_under)
print(strategy_under, sep='\n')

data_under, e_under=under.fit_resample(data, e)
print('Undersampled dataset shape %s' % Counter(e_under))

Original dataset shape Counter({0: 1278, 1: 454, 3: 211, 2: 206})
{0: 206, 1: 206, 2: 206, 3: 206}
Undersampled dataset shape Counter({0: 206, 1: 206, 2: 206, 3: 206})


### Split the data

In [4]:
X, y = data_under.drop("Diagnosis", axis=1), data_under["Diagnosis"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=22, stratify=y)
X_train.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,MMSE,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness
1042,5793,62,1,3,0,16.522671,0,10.353125,9.300418,8.165480,...,5.901941,0.482852,0,0,1.016881,0,0,0,0,1
1686,6437,60,1,2,1,38.641313,1,3.938894,3.276951,1.187660,...,11.696364,6.120013,0,1,0.134678,0,0,1,0,0
1352,6103,63,0,3,1,23.356043,1,17.567740,8.565534,8.233385,...,0.364238,1.466079,0,0,1.391462,0,0,0,0,0
276,5027,75,0,2,2,26.971268,0,9.571022,0.009348,4.597253,...,2.065666,9.761932,0,0,7.470467,1,0,1,1,0
505,5256,73,0,2,1,20.260631,0,1.144111,7.370079,7.576394,...,17.972518,7.238521,1,0,0.212521,0,0,0,1,1


### Train the model

In [5]:
classifier = MLPClassifier(max_iter=5000, random_state=12)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [6]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.63      0.66       133
           1       0.43      0.51      0.47        73

    accuracy                           0.59       206
   macro avg       0.57      0.57      0.56       206
weighted avg       0.60      0.59      0.59       206



In [7]:
X["Ethnicity"].value_counts()

Ethnicity
0    206
1    206
2    206
3    206
Name: count, dtype: int64

### Compare performance based on ethnicity

In [8]:
# Join X_test, y_test, y_pred for analysis
results = X_test.copy()
results["TrueDiagnosis"] = y_test
results["PredictedDiagnosis"] = y_pred

In [9]:
# Ethnicity 0
eth0_results = results[results["Ethnicity"] == 0]
print(classification_report(eth0_results["TrueDiagnosis"], eth0_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.65      0.61      0.62        33
           1       0.38      0.42      0.40        19

    accuracy                           0.54        52
   macro avg       0.51      0.51      0.51        52
weighted avg       0.55      0.54      0.54        52



In [10]:
# Ethncity 1
eth1_results = results[results["Ethnicity"] == 1]
print(classification_report(eth1_results["TrueDiagnosis"], eth1_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.79      0.63      0.70        35
           1       0.35      0.54      0.42        13

    accuracy                           0.60        48
   macro avg       0.57      0.58      0.56        48
weighted avg       0.67      0.60      0.62        48



In [11]:
# Ethnicity 2  
eth2_results = results[results["Ethnicity"] == 2]
print(classification_report(eth2_results["TrueDiagnosis"], eth2_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.62      0.74      0.68        27
           1       0.61      0.48      0.54        23

    accuracy                           0.62        50
   macro avg       0.62      0.61      0.61        50
weighted avg       0.62      0.62      0.61        50



In [12]:
# Ethncity 3
eth3_results = results[results["Ethnicity"] == 3]
print(classification_report(eth3_results["TrueDiagnosis"], eth3_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.76      0.58      0.66        38
           1       0.41      0.61      0.49        18

    accuracy                           0.59        56
   macro avg       0.58      0.60      0.57        56
weighted avg       0.65      0.59      0.60        56



In [16]:
from fairness_metrics import demographic_parity, equal_opportunity, disparate_impact

# DI >= 0.8 is a pre-established threshold for fairness
# DP and EO need to be as close to 0 as possible

dp = demographic_parity(results, 0, 1)
eo = equal_opportunity(results, 0, 1)
di = disparate_impact(results, 0, 1)

print(dp, eo, di)

from fairness_metrics import demographic_parity, equal_opportunity, disparate_impact

# DI >= 0.8 is a pre-established threshold for fairness
# DP and EO need to be as close to 0 as possible

dp = demographic_parity(results, 0, 2)
eo = equal_opportunity(results, 0, 2)
di = disparate_impact(results, 0, 2)

print(dp, eo, di)


from fairness_metrics import demographic_parity, equal_opportunity, disparate_impact

# DI >= 0.8 is a pre-established threshold for fairness
# DP and EO need to be as close to 0 as possible

dp = demographic_parity(results, 0, 3)
eo = equal_opportunity(results, 0, 3)
di = disparate_impact(results, 0, 3)

print(dp, eo, di)

0.01282051282051283 0.030952380952380953 0.9692307692307692
0.04384615384615387 0.23015873015873023 1.1217948717948718
0.0782967032967033 0.026455026455026454 0.8376068376068376
